# Imports

In [2]:
import numpy as np
import os
import pytest
from types import SimpleNamespace
import ipywidgets as widgets

from library.pysolver.solver import *
from library.model.model import *
import library.model.initial_conditions as IC
import library.model.boundary_conditions as BC
from library.pysolver.ode import RK1
import library.misc.io as io
from library.pysolver.reconstruction import GradientMesh
import library.postprocessing.postprocessing as postprocessing

In [ ]:

    settings = Settings(
        name="ShallowMoments2d",
        momentum_eqns=[1, 2] + [3 + l for l in range(2 * level)],
        parameters={"g": 1.0, "C": 16.0, "nu": 0.1},
        reconstruction=recon.constant,
        num_flux=flux.LLF(),
        compute_dt=timestepping.adaptive(CFL=0.45),
        time_end=1.0,
        output_snapshots=30,
        output_dir='output_test'
    )


In [ ]:

    main_dir = os.getenv("SMS")
    mesh = Mesh.load_gmsh(
        os.path.join(main_dir, "meshes/{}_2d/mesh_coarse.msh".format('triangle')),
        'triangle',
    )


In [ ]:

    bc_tags = ["left", "right", "top", "bottom"]
    bc_tags_periodic_to = ["right", "left", "bottom", "top"]

    bcs = BC.BoundaryConditions(
        [
            BC.Periodic(physical_tag=tag, periodic_to_physical_tag=tag_periodic_to)
            for (tag, tag_periodic_to) in zip(bc_tags, bc_tags_periodic_to)
        ]
    )

In [ ]:

    ic = IC.RP(
        left=lambda n_field: np.array(
            [2.0, 0.0, 0.0] + [0.0 for l in range(2 * level)]
        ),
        right=lambda n_field: np.array(
            [1.0, 0.0, 0.0] + [0.0 for l in range(2 * level)]
        ),
    )

In [ ]:


    level = 2
    model = ShallowMoments2d(
        dimension=2,
        fields=3 + 2 * level,
        aux_fields=0,
        parameters=settings.parameters,
        boundary_conditions=bcs,
        initial_conditions=ic,
        settings={"friction": ["chezy"]},
    )

In [ ]:

    fvm_unsteady_semidiscrete(mesh, model, settings, RK1)

In [ ]:

    mesh = GradientMesh.fromMesh(mesh)

    io.generate_vtk(settings.output_dir)
    # postprocessing.recover_3d_from_smm_as_vtk(
    #     model,
    #     settings.output_dir,
    #     os.path.join(settings.output_dir, "mesh.hdf5"),
    #     os.path.join(settings.output_dir, "fields.hdf5"),
    #     Nz=10,
    #     start_at_time=1.0,
    # )

In [4]:
accordion = widgets.Accordion(children=[widgets.IntSlider(), widgets.Text()], titles=('Slider', 'Text'))
tab_nest = widgets.Tab()
tab_nest.children = [accordion, accordion]
tab_nest.titles = ('An accordion', 'Copy of the accordion')


In [3]:
accordion

Accordion(children=(IntSlider(value=0), Text(value='')), titles=('Slider', 'Text'))

In [5]:
tab_nest

In [11]:
from ipyfilechooser import FileChooser

main_dir = os.getenv("SMS")
default_mesh_path = os.path.join(main_dir, "meshes/quad_2d")
# Create and display a FileChooser widget
fc = FileChooser(default_mesh_path)
fc.default_path = default_mesh_path
fc.default_filename = 'mesh_coarse.msh'
fc.filter_pattern = '*.msh'
fc.reset()
display(fc)


FileChooser(path='/home/ingo/git/shallow-moments-simulation/meshes/quad_2d', filename='mesh_coarse.msh', title…

In [16]:
# load mesh with pyvista
# extract physical tags from mesh (loadmesh association table has that I think)
# but I do not want to load the whole mesh yet...
# mesh stats?
# combine in one tab
# callbacks, such that chaning the value automatically triggers a recalculation
